# Train

In [1]:
import os
import json
import time
import torch
import argparse
import numpy as np
from multiprocessing import cpu_count
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from collections import OrderedDict, defaultdict

from ptb import PTB
from utils import to_var, idx2word, experiment_name, AttributeDict
from model_kwbase import SentenceVAE
from glob import glob

In [3]:
# data_dir = 'data/simple-examples'
base_dir = 'data/exp_20191220_kw2title'
# data_name = 'ec.mediatag'
data_name = 'hr.mediatag'
# data_name = 'finance.categorytag'
# data_name = 'finance.mediatag'
data_dir = f'{base_dir}/{data_name}'

In [4]:
def readlines(path):
    with open(path, 'r') as f:
        return [s.replace('\n', '') for s in f.readlines()]

def cal_max_file_lines(path):
    lines = readlines(path)
    line_lengths = [len(line.split(' ')) for line in lines]
    return max(line_lengths)
    
src_max_length = cal_max_file_lines(f'{data_dir}/src/ptb.train.txt')
tgt_max_length = cal_max_file_lines(f'{data_dir}/tgt/ptb.train.txt')
print(f'src max: {src_max_length}, tgt max: {tgt_max_length}')

src max: 5, tgt max: 50


In [5]:
args = {
    'data_dir': data_dir,
    'create_data': False,
    'max_sequence_length': tgt_max_length,
    'max_sequence_length_src': src_max_length,
    'min_occ': 1,
    'test': False,

    'epochs': 10,
    'batch_size': 32,
    'learning_rate': 0.001,

    'embedding_size': 300,
    'rnn_type': 'gru',
    'hidden_size': 256,
    'num_layers': 1,
    'bidirectional': False,
    'latent_size': 16,
    'word_dropout': 0,
    'embedding_dropout': 0.5,

    'anneal_function': 'logistic',
    'k': 0.0025,
    'x0': 2500,

    'print_every': 50,
    'tensorboard_logging': False,
    'logdir': '/root/user/work/logs/',
    'save_model_path': 'bin',
    'experiment_name': f'kw2copy_{data_name}',
}

args = AttributeDict(args)

args.rnn_type = args.rnn_type.lower()
args.anneal_function = args.anneal_function.lower()

assert args.rnn_type in ['rnn', 'lstm', 'gru']
assert args.anneal_function in ['logistic', 'linear']
assert 0 <= args.word_dropout <= 1
args

<AttrDict{'data_dir': 'data/exp_20191220_kw2title/hr.mediatag', 'create_data': False, 'max_sequence_length': 50, 'max_sequence_length_src': 5, 'min_occ': 1, 'test': False, 'epochs': 10, 'batch_size': 32, 'learning_rate': 0.001, 'embedding_size': 300, 'rnn_type': 'gru', 'hidden_size': 256, 'num_layers': 1, 'bidirectional': False, 'latent_size': 16, 'word_dropout': 0, 'embedding_dropout': 0.5, 'anneal_function': 'logistic', 'k': 0.0025, 'x0': 2500, 'print_every': 50, 'tensorboard_logging': False, 'logdir': '/root/user/work/logs/', 'save_model_path': 'bin', 'experiment_name': 'kw2copy_hr.mediatag'}>

## load data

In [6]:
%%time
import itertools
splits = ['train', 'valid'] + (['test'] if args.test else [])
datasets = OrderedDict()
print(f'loading {args.data_dir}')
for split, src_tgt in itertools.product(splits, ['src', 'tgt']):
    key = (split, src_tgt)
    print(key)
    datasets[key] = PTB(
        data_dir=f'{args.data_dir}/{src_tgt}',
        split=split,
        create_data=args.create_data,
        max_sequence_length=args.max_sequence_length if src_tgt == 'tgt' else args.max_sequence_length_src,
        min_occ=args.min_occ
    )
    print(f'vocab: {datasets[key].vocab_size}, records: {len(datasets[key].data)}')

loading data/exp_20191220_kw2title/hr.mediatag
('train', 'src')
vocab: 1235, records: 17919
('train', 'tgt')
vocab: 5622, records: 17919
('valid', 'src')
vocab: 1235, records: 110
('valid', 'tgt')
vocab: 5622, records: 110
CPU times: user 267 ms, sys: 26 ms, total: 293 ms
Wall time: 295 ms


In [7]:
# 実際のデータ確認
def ids2text(id_list, ptb):
    return ' '.join([ptb.i2w[f'{i}'] for i in id_list])

_ptb_src = datasets[('train', 'src')]
_ptb_tgt = datasets[('train', 'tgt')]
index = str(101)
_sample_src, _sample_tgt = _ptb_src.data[index], _ptb_tgt[index]
print(f'■ src-input \n{ids2text(_sample_src["input"], _ptb_src)}')
print(f'■ src-target \n{ids2text(_sample_src["target"], _ptb_src)}')
print(f'■ tgt-input\n{ids2text(_sample_tgt["input"], _ptb_tgt)}')
print(f'■ tgt-target\n{ids2text(_sample_tgt["target"], _ptb_tgt)}')

■ src-input 
<sos> other <sgltag> 申し込み ai
■ src-target 
other <sgltag> 申し込み ai <eos>
■ tgt-input
<sos> ai 、 機械 学習 チャレンジ や <org> の 申込み を 受付中 ! <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
■ tgt-target
ai 、 機械 学習 チャレンジ や <org> の 申込み を 受付中 ! <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


## build model

In [8]:
from ptb import SOS_INDEX, EOS_INDEX, PAD_INDEX, UNK_INDEX

In [9]:
%load_ext autoreload

In [10]:
%autoreload
model = SentenceVAE(
    vocab_size=datasets[('train', 'src')].vocab_size,
    sos_idx=SOS_INDEX,
    eos_idx=EOS_INDEX,
    pad_idx=PAD_INDEX,
    unk_idx=UNK_INDEX,
    max_sequence_length=args.max_sequence_length,
    embedding_size=args.embedding_size,
    rnn_type=args.rnn_type,
    hidden_size=args.hidden_size,
    word_dropout=args.word_dropout,
    embedding_dropout=args.embedding_dropout,
    latent_size=args.latent_size,
    num_layers=args.num_layers,
    bidirectional=args.bidirectional,
    
    # bow loss
    # bow_hidden_size=256,
    use_bow_loss=False,
    
    # kw base
    out_vocab_size=datasets[('train', 'tgt')].vocab_size,
    )

if torch.cuda.is_available():
    model = model.cuda()

In [11]:
model

SentenceVAE(
  (embedding): Embedding(1235, 300)
  (embedding_dropout): Dropout(p=0.5, inplace=False)
  (decoder_embedding): Embedding(5622, 300)
  (encoder_rnn): GRU(300, 256, batch_first=True)
  (decoder_rnn): GRU(300, 256, batch_first=True)
  (hidden2mean): Linear(in_features=256, out_features=16, bias=True)
  (hidden2logv): Linear(in_features=256, out_features=16, bias=True)
  (latent2hidden): Linear(in_features=16, out_features=256, bias=True)
  (outputs2vocab): Linear(in_features=256, out_features=5622, bias=True)
)

## log

In [12]:
def get_meta_model_dict(model, args):
    meta_dict = {k:v for k, v in model.__dict__.items() if not k[0] == '_'}
    meta_dict.update(args.obj)
    return meta_dict

In [13]:
print(f'tensorboard logging: {args.tensorboard_logging}')
ts = time.strftime('%Y-%b-%d-%H:%M:%S', time.gmtime())
exp_name = experiment_name(args,ts)

if args.tensorboard_logging:
    writer_path = os.path.join(args.logdir, exp_name)
    writer = SummaryWriter(writer_path)
    writer.add_text("model", str(model))
    writer.add_text("args", str(args))
    writer.add_text("ts", ts)
    print(f'tensorboard logging → {writer_path}')
    
save_model_path = os.path.join(args.save_model_path, exp_name)
os.makedirs(save_model_path)
print(f'▼ model save\n{save_model_path}')

# メタパラメータ保存
with open(os.path.join(save_model_path, 'model_meta.json'), 'w') as f:
    meta_dict = get_meta_model_dict(model, args)
    meta_dict.pop('tensor')
    json.dump(meta_dict, f)

tensorboard logging: False
▼ model save
bin/kw2copy_hr.mediatag_BS=32_LR=0.001_EB=300_GRU_HS=256_L=1_BI=0_LS=16_WD=0_ANN=LOGISTIC_K=0.0025_X0=2500_TS=2019-Dec-23-05:54:04


## optimizer

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
step = 0

In [15]:
datasets.keys()

odict_keys([('train', 'src'), ('train', 'tgt'), ('valid', 'src'), ('valid', 'tgt')])

In [16]:
ae_datasets = {split: dataset for (split, src_tgt), dataset in datasets.items() if src_tgt == 'tgt'}
print(ids2text(ae_datasets['train'][0]['input'], ae_datasets['train']))
print(ids2text(ae_datasets['train'][0]['target'], ae_datasets['train']))

<sos> 「 これ から 就活 準備 を する あなた へ 」 リクナビ <num> で 【 <org> 診断 】 や 【 spi 性格 検査 】 を 使っ て 自己 理解 を 深め て 就活 準備 に 役立てよう ! <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
「 これ から 就活 準備 を する あなた へ 」 リクナビ <num> で 【 <org> 診断 】 や 【 spi 性格 検査 】 を 使っ て 自己 理解 を 深め て 就活 準備 に 役立てよう ! <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [17]:
_datasets = {}
for split in splits:
    src_dataset = datasets[(split, 'src')]
    tgt_dataset = datasets[(split, 'tgt')]
    assert len(src_dataset) == len(tgt_dataset)
    dataset = []
    for i in range(len(src_dataset)):
        src_set, tgt_set = src_dataset[i], tgt_dataset[i]
        _data = {}
        _data.update({f'src_{k}': v for k,v in src_set.items()})
        _data.update({f'tgt_{k}': v for k,v in tgt_set.items()})
        dataset.append(_data)
    _datasets[split] = dataset

In [18]:
src_set

{'input': array([  2,   9,   5, 125, 488]),
 'target': array([  9,   5, 125, 488,   3]),
 'length': 5}

In [19]:
_datasets.keys()

dict_keys(['train', 'valid'])

In [20]:
_datasets['train'][0]

{'src_input': array([2, 4, 5, 6, 7]),
 'src_target': array([4, 5, 6, 7, 3]),
 'src_length': 5,
 'tgt_input': array([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 17, 22, 23, 24, 20,  9, 25, 26, 27, 28,  9, 29, 26,  7,  8,
        30, 31, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 'tgt_target': array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 17, 22, 23, 24, 20,  9, 25, 26, 27, 28,  9, 29, 26,  7,  8, 30,
        31, 32,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 'tgt_length': 37}

In [21]:
train_target_ptb = datasets[('train', 'tgt')]
train_target_ptb

In [22]:
from utils import ids2ptext

In [23]:
%pdb on
for epoch in range(args.epochs):

    for split in splits:
        
        data_loader = DataLoader(
            dataset=_datasets[split],
            batch_size=args.batch_size,
            shuffle=split=='train',
            num_workers=cpu_count(),
            pin_memory=torch.cuda.is_available()
        )

        tracker = defaultdict(tensor)

        # Enable/Disable Dropout
        if split == 'train':
            model.train()
        else:
            model.eval()

        for iteration, batch in enumerate(data_loader):
            
            batch_size = batch['src_input'].size(0)
            
            for k, v in batch.items():
                if torch.is_tensor(v):
                    batch[k] = to_var(v)
            
            # loss calculation
            cal_dict = model(batch['src_input'], batch['src_length'], batch['tgt_input'], batch['tgt_length'])
            logp, mean, logv, z = cal_dict['logp'], cal_dict['mean'], cal_dict['logv'], cal_dict['z']
            
            loss_dict = model.loss(logp, batch['tgt_target'], batch['tgt_length'], mean, logv, args.anneal_function, step, args.k, args.x0, bow_input=z)
            loss, NLL_loss, KL_weight, KL_loss, avg_bow_loss = loss_dict['loss'], loss_dict['NLL_loss'], loss_dict['KL_weight'], loss_dict['KL_loss'], loss_dict.get('avg_bow_loss')

            # backward + optimization
            if split == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                step += 1

            # bookkeepeing
            tracker['ELBO'] = torch.cat((tracker['ELBO'], loss.data.view(1)))

            if args.tensorboard_logging:
                writer.add_scalar("%s/ELBO"%split.upper(), loss.data.item(), epoch*len(data_loader) + iteration)
                writer.add_scalar("%s/NLL Loss"%split.upper(), NLL_loss.data.item()/batch_size, epoch*len(data_loader) + iteration)
                writer.add_scalar("%s/KL Loss"%split.upper(), KL_loss.data.item()/batch_size, epoch*len(data_loader) + iteration)
                writer.add_scalar("%s/KL Weight"%split.upper(), KL_weight, epoch*len(data_loader) + iteration)
                if avg_bow_loss is not None:
                    writer.add_scalar("%s/BOW Loss"%split.upper(), avg_bow_loss, epoch*len(data_loader) + iteration)

            if iteration % args.print_every == 0 or iteration+1 == len(data_loader):
                print_text = "%s Batch %04d/%i, Loss %9.4f, NLL-Loss %9.4f, KL-Loss %9.4f, KL-Weight %6.3f"%(split.upper(), iteration, len(data_loader)-1, loss.data.item(), NLL_loss.data.item()/batch_size, KL_loss.data.item()/batch_size, KL_weight)
                if avg_bow_loss is not None:
                    print_text += ', BOW Loss %9.4f,'%(avg_bow_loss)
                print(print_text)

            if split == 'valid':
                if 'target_sents' not in tracker:
                    tracker['target_sents'] = list()
                tracker['target_sents'] += idx2word(batch['tgt_target'].data, i2w=train_target_ptb.get_i2w(), pad_idx=PAD_INDEX)
                tracker['z'] = torch.cat((tracker['z'], z.data), dim=0)

        print("%s Epoch %02d/%i, Mean ELBO %9.4f"%(split.upper(), epoch, args.epochs, torch.mean(tracker['ELBO'])))

        if args.tensorboard_logging:
            writer.add_scalar("%s-Epoch/ELBO"%split.upper(), torch.mean(tracker['ELBO']), epoch)

        # save a dump of all sentences and the encoded latent space
        if split == 'valid':
            dump = {'target_sents':tracker['target_sents'], 'z':tracker['z'].tolist()}
            if not os.path.exists(os.path.join('dumps', ts)):
                os.makedirs('dumps/'+ts)
            with open(os.path.join('dumps/'+ts+'/valid_E%i.json'%epoch), 'w') as dump_file:
                json.dump(dump,dump_file)
           
        # TODO: テストデータのログ
        # if split == 'test':
            # 1. テストデータ全体に対してデコード
            # 2. 各種指標の計測
            # 3. デコードサンプルをtensorboard TEXTに保存
            # decoded_ids, _ = model.inference(z=z)
            # decoded_texts = [ids2ptext(text_ids.squeeze(), train_target_ptb.i2w) for text_ids in decoded_ids]

        # save checkpoint
        if split == 'train':
            checkpoint_path = os.path.join(save_model_path, "E%i.pytorch"%(epoch))
            torch.save(model.state_dict(), checkpoint_path)
            print("Model saved at %s"%checkpoint_path)

Automatic pdb calling has been turned ON
TRAIN Batch 0000/559, Loss  171.7726, NLL-Loss  171.7719, KL-Loss    0.3656, KL-Weight  0.002
TRAIN Batch 0050/559, Loss  115.6721, NLL-Loss  115.6223, KL-Loss   22.7999, KL-Weight  0.002
TRAIN Batch 0100/559, Loss  111.4237, NLL-Loss  111.3514, KL-Loss   29.2286, KL-Weight  0.002
TRAIN Batch 0150/559, Loss   96.3600, NLL-Loss   96.2723, KL-Loss   31.2987, KL-Weight  0.003


KeyboardInterrupt: 

> /Users/s07309/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/autograd/__init__.py(99)backward()
     97     Variable._execution_engine.run_backward(
     98         tensors, grad_tensors, retain_graph, create_graph,
---> 99         allow_unreachable=True)  # allow_unreachable flag
    100 
    101 

ipdb> ｑ
*** NameError: name 'q' is not defined
ipdb> q


- TEST時にやりたいこと
    - 評価指標による計測
    - デコードをTEXTに保存
- 評価指標の追加：TrainingのTEST時に評価
    - BLEU
    - Distinct-1, 2, full
    - Train_contains_decode